In [ ]:
import cv2
import matplotlib.pyplot as plt

# Load the images
im0 = cv2.imread('/mnt/data/im0.png', cv2.IMREAD_GRAYSCALE)
im1 = cv2.imread('/mnt/data/im1.png', cv2.IMREAD_GRAYSCALE)

# Visualize the images
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(im0, cmap='gray')
axes[0].set_title('Left Image (im0.png)')
axes[0].axis('off')
axes[1].imshow(im1, cmap='gray')
axes[1].set_title('Right Image (im1.png)')
axes[1].axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# Set up the StereoSGBM matcher with the provided parameters again
stereo = cv2.StereoSGBM_create(
    minDisparity=vmin,
    numDisparities=ndisp,
    blockSize=16,
    P1=8*3*3,
    P2=32*3*3,
    disp12MaxDiff=1,
    uniquenessRatio=10,
    speckleWindowSize=100,
    speckleRange=32,
)

# Compute the disparity map again
disparity = stereo.compute(im0, im1)

# Visualize the disparity map
plt.figure(figsize=(8, 6))
plt.imshow(disparity, cmap='jet')
plt.colorbar()
plt.title('Disparity Map')
plt.axis('off')
plt.show()


In [ ]:
import numpy as np

# Extracting the focal length and principal point from the camera matrix
f = cam0[0][0]  # focal length
cx = cam0[0][2]  # principal point x-coordinate
cy = cam0[1][2]  # principal point y-coordinate

# Create a meshgrid of pixel coordinates
y, x = np.mgrid[0:height, 0:width]

# Compute 3D points
Z = (f * baseline) / (disparity + 1e-6)  # Adding a small value to avoid division by zero
X = (x - cx) * Z / f
Y = (y - cy) * Z / f

# Filtering out invalid points (where disparity <= 0)
mask = disparity > 0
X = X[mask]
Y = Y[mask]
Z = Z[mask]

# Stacking the 3D points into a single matrix
points_3D = np.vstack((X, Y, Z)).T

points_3D.shape


In [ ]:
# Load the left image in color
im0_color = cv2.imread('/mnt/data/im0.png', cv2.IMREAD_COLOR)
im0_color = cv2.cvtColor(im0_color, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

# Extract RGB values for the valid 3D points
colors = im0_color[y[mask], x[mask]]

# Visualizing the point cloud using matplotlib
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
subset = np.random.choice(points_3D.shape[0], 10000, replace=False)  # Visualizing a subset for efficiency
ax.scatter(points_3D[subset, 0], points_3D[subset, 1], points_3D[subset, 2], c=colors[subset] / 255.0, s=1)
ax.set_title('Colored Point Cloud')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=45, azim=-90)  # Adjusting the view angle for better visualization
plt.show()

# Saving the point cloud to a .ply file
ply_filename = '/mnt/data/point_cloud.ply'
with open(ply_filename, 'w') as f:
    # PLY header
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write(f"element vertex {len(points_3D)}\n")
    f.write("property float x\n")
    f.write("property float y\n")
    f.write("property float z\n")
    f.write("property uchar red\n")
    f.write("property uchar green\n")
    f.write("property uchar blue\n")
    f.write("end_header\n")
    # PLY data
    for point, color in zip(points_3D, colors):
        f.write(f"{point[0]} {point[1]} {point[2]} {color[0]} {color[1]} {color[2]}\n")

ply_filename


In [ ]:
# Updating the provided parameters for the new images
cam0_new = [[1758.23, 0, 872.36], [0, 1758.23, 552.32], [0, 0, 1]]
cam1_new = [[1758.23, 0, 872.36], [0, 1758.23, 552.32], [0, 0, 1]]
doffs_new = 0
baseline_new = 124.86
ndisp_new = 310
vmin_new = 90
vmax_new = 280

# Set up the StereoSGBM matcher with the new parameters
stereo_new = cv2.StereoSGBM_create(
    minDisparity=vmin_new,
    numDisparities=ndisp_new,
    blockSize=16,
    P1=8*3*3,
    P2=32*3*3,
    disp12MaxDiff=1,
    uniquenessRatio=10,
    speckleWindowSize=100,
    speckleRange=32,
)

# Compute the disparity map for the new images
disparity_new = stereo_new.compute(im0_new, im1_new)

# Visualize the new disparity map
plt.figure(figsize=(8, 6))
plt.imshow(disparity_new, cmap='jet')
plt.colorbar()
plt.title('New Disparity Map')
plt.axis('off')
plt.show()


In [ ]:
# Extracting the focal length and principal point from the new camera matrix
f_new = cam0_new[0][0]  # focal length
cx_new = cam0_new[0][2]  # principal point x-coordinate
cy_new = cam0_new[1][2]  # principal point y-coordinate

# Create a meshgrid of pixel coordinates for the new images
y_new, x_new = np.mgrid[0:height, 0:width]

# Compute 3D points for the new disparity map
Z_new = (f_new * baseline_new) / (disparity_new + 1e-6)  # Adding a small value to avoid division by zero
X_new = (x_new - cx_new) * Z_new / f_new
Y_new = (y_new - cy_new) * Z_new / f_new

# Filtering out invalid points (where disparity <= 0)
mask_new = disparity_new > 0
X_new = X_new[mask_new]
Y_new = Y_new[mask_new]
Z_new = Z_new[mask_new]

# Load the new left image in color for extracting RGB values
im0_color_new = cv2.imread('/mnt/data/im0.png', cv2.IMREAD_COLOR)
im0_color_new = cv2.cvtColor(im0_color_new, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

# Extract RGB values for the valid 3D points from the new images
colors_new = im0_color_new[y_new[mask_new], x_new[mask_new]]

# Visualizing the point cloud using matplotlib
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
subset_new = np.random.choice(X_new.shape[0], 10000, replace=False)  # Visualizing a subset for efficiency
ax.scatter(X_new[subset_new], Y_new[subset_new], Z_new[subset_new], c=colors_new[subset_new] / 255.0, s=1)
ax.set_title('New Colored Point Cloud')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=45, azim=-90)  # Adjusting the view angle for better visualization
plt.show()

# Saving the new point cloud to a .ply file
ply_filename_new = '/mnt/data/point_cloud_new.ply'
with open(ply_filename_new, 'w') as f:
    # PLY header
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write(f"element vertex {len(X_new)}\n")
    f.write("property float x\n")
    f.write("property float y\n")
    f.write("property float z\n")
    f.write("property uchar red\n")
    f.write("property uchar green\n")
    f.write("property uchar blue\n")
    f.write("end_header\n")
    # PLY data
    for point, color in zip(zip(X_new, Y_new, Z_new), colors_new):
        f.write(f"{point[0]} {point[1]} {point[2]} {color[0]} {color[1]} {color[2]}\n")

ply_filename_new
